## Общая информация по проекту.

Цель проекта: изучение принципов работы алгоритмов рекомендательных систем, создание прототипа.

Задачи проекта:

- Реализация EDA/FE

- Использование след. моделей:  LightFM, SVD, BaselineOnly(ALS/SGD)

- Создание прототипа

- Получить выводы и дальнейшие рекомендации


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_profiling # import profile for EDA


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are avaHBVCilable in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Any results you write to the current directory are saved as output.
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import surprise
from surprise import Reader, Dataset

from lightfm.cross_validation import random_train_test_split

import scipy.sparse as sparse

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
import sklearn
from sklearn.model_selection import train_test_split

import scipy.sparse as sparse


from surprise import SVD, AlgoBase, KNNBasic
from surprise import Reader, Dataset
from surprise.model_selection import GridSearchCV

from surprise import BaselineOnly

from surprise import accuracy
from surprise.model_selection import PredefinedKFold

from lightfm import LightFM, cross_validation



In [ ]:
train = pd.read_csv('/kaggle/input/recommendationsv4/train.csv')
test = pd.read_csv('/kaggle/input/recommendationsv4/test.csv')
submission = pd.read_csv('/kaggle/input/recommendationsv4/sample_submission.csv')

In [ ]:
import json

with open('/kaggle/input/recommendationsv4/meta_Grocery_and_Gourmet_Food.json') as f:
    meta_list = []
    for line in f.readlines():
        meta_list.append(json.loads(line))
        
meta = pd.DataFrame(meta_list)

In [ ]:
#склеим данные по признаку asin
dataset = pd.merge(train, meta, on='asin')

# EDA

In [ ]:
meta.info()

In [ ]:
meta.sample(5)

In [ ]:
train.shape, test.shape, submission.shape, dataset.shape

In [ ]:
dataset.sample(5)

In [ ]:
train.sample(5)

In [ ]:
dataset.isna().sum()

    Тестовый и тернировочный дадасеты отличаются по размеру, удалим из train столбец summary
    Удалим столбцы, которые вероятно не являются информативными: asin, vote, style, image_y, details 
    Удалми столбцы, имеющие много пропусков:  vote, style,image_x, also_view, price, also_buy, date, feature, fit,tech, similar_item 
    Удалим дублирующие столбцы: reviewerName, reviewTime 
    Удалим признак reviewText и Description  данные признаки могут быть информативными(но не хватает времени на  обработку)

In [ ]:
#удалим лишние признаки
train_json = dataset.drop(['vote', 'style','image_x', 'also_view', 'price', 'also_buy', 'date', 'feature', 'fit','tech1', 'similar_item','reviewerName', 'reviewTime','description','reviewText', 'vote', 'style', 'image_y','details','summary','asin'], axis=1) 

In [ ]:
#оцифуем столбец verifed
dic_verified = {
    True: 1,
    False: 0
}
train_json['verified'] = train_json['verified'].map(dic_verified)

In [ ]:
# поработаем с признаком unixReviewTime, предположительно дата отзыва может влиять 
from datetime import datetime
tsmin = train_json.unixReviewTime.min()
tsmax = train_json.unixReviewTime.max()
date_25 = int(train_json.unixReviewTime.quantile(0.25))
date_50 = int(train_json.unixReviewTime.quantile(0.50))
date_75 = int(train_json.unixReviewTime.quantile(0.75))
datetime.utcfromtimestamp(tsmin).strftime('%Y-%m-%d %H:%M:%S'), datetime.utcfromtimestamp(tsmax).strftime('%Y-%m-%d %H:%M:%S'),datetime.utcfromtimestamp(date_75).strftime('%Y-%m-%d %H:%M:%S'), datetime.utcfromtimestamp(date_25).strftime('%Y-%m-%d %H:%M:%S'),datetime.utcfromtimestamp(date_50).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
#получили разбег по времени почти 18 лет, данный признак буд оцифровывать по следующему принципу квантилей(идею заимствовал)
def digitdate(date):
    if date <= date_25: date = 1
    elif date_25 < date <= date_50: date = 2
    elif date_50 < date <= date_75: date = 3
    elif date_75 < date: date = 4
    return date      

train_json['unixReviewTime'] = train_json['unixReviewTime'].apply(lambda x: digitdate(x))


In [ ]:
#предобработка датасета завершена, теперь посмотрим графику, корелляции и тд 
#profile = pandas_profiling.ProfileReport(train)
#profile.to_file("rec_sys.html")
#profile

#### unixReviewTime "условно хорошо" коррелирует с itemid
#### verified "условно хорошо" коррелирует с itemid


In [ ]:
#создадим дамми перменные, предварительно создав категорию other
brand_freqs = list(train_json.brand.value_counts())
top_brand_count = int(np.percentile(brand_freqs, 90))
all_brand = train_json.brand.value_counts().index
top_brand = list(all_brand)[:top_brand_count]
brand_to_throw_away = list(set(all_brand) - set(top_brand))
train_json.loc[train_json.brand.isin(brand_to_throw_away),'brand'] = 'other'

train_json = pd.get_dummies(train_json, columns=['brand'], dummy_na=True)


# FE

### Подготовим доп признаки для метода fit¶


In [ ]:
# подготовим данные
features_user = train_json[['userid', 'verified']]
features_item = train_json[['itemid', 'unixReviewTime']]
train_json_fe = train_json[['userid','itemid','rating']]

In [ ]:
# используем unixReviewTime, brand, для item_features
# используем verified для user_features
# сформируем списки для item_features формата 'feature_name: feature_value'

item_feat = []
name = []
value = []
for column in features_item.drop(['itemid'], axis=1):
    name += [column]*len(features_item[column].unique())
    value += list(features_item[column].unique())
for x,y in zip(name, value):
    res = str(x)+ ":" +str(y)
    item_feat.append(res)
  


In [ ]:
# сформируем списки для user_features формата 'feature_name: feature_value'

user_feat = []
name = []
value = []
for column in features_user.drop(['userid'], axis=1):
    name += [column]*len(features_user[column].unique())
    value += list(features_user[column].unique())
for x,y in zip(name, value):
    res = str(x)+ ":" +str(y)
    user_feat.append(res)
    

In [ ]:
from lightfm.data import Dataset
# формируем датасет методом fit

dataset_features = Dataset()
dataset_features.fit(train_json_fe['userid'].unique(),train_json_fe['itemid'].unique(),user_features = user_feat,item_features = item_feat)

In [ ]:
# добавляем взаимодействия(interactions) используя метод build_item_features 
# создадим список необходимого для  build_item_features формата

list_for_build = []
for column in features_item.drop(['itemid'], axis=1):
    list_for_build.append(column + ':')

def feature_value(my_list):
    result = []
    a = my_list
    for x,y in zip(list_for_build,a):
        res = str(x) +""+ str(y)
        result.append(res)
    return result

ad_subset = features_item.drop(['itemid'], axis=1)
ad_list = [x.tolist() for x in ad_subset.values]
item_feature_list = []

for item in ad_list:
    item_feature_list.append(feature_value(item))



In [ ]:
item_tuple = list(zip(features_item.itemid, item_feature_list))

item_features = dataset_features.build_item_features(item_tuple, normalize= False)
item_features.todense()

In [ ]:
#В итоге мы получили матрицу взаимодействий, где строки - это продукты, а столбцы - это фичи продуктов

In [ ]:
# аналогично сделаем для user_features
list_for_build = []
for column in features_user.drop(['userid'], axis=1):
    list_for_build.append(column + ':')

ad_subset = features_user.drop(['userid'], axis=1)
ad_list = [x.tolist() for x in ad_subset.values]
user_feature_list = []

for user in ad_list:
    user_feature_list.append(feature_value(user))

In [ ]:
user_tuple = list(zip(features_user.userid, user_feature_list))
user_features = dataset_features.build_user_features(user_tuple, normalize= False)
user_features.todense()


# ML

In [ ]:
# для использования FE создадим словари с помощью которых по id в датасете LightFM мы сможем находить id в датасете train_json
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset_features.mapping()
(interactions, weights) = dataset_features.build_interactions([(x[0], x[1], x[2]) for x in train_json_fe.values])


In [ ]:
#LightFM with FE
model_lfm_fe = LightFM(loss='warp')
model_lfm_fe.fit(interactions, 
    user_features = user_features,
    item_features = item_features, 
    sample_weight = weights, 
    epochs=10)

In [ ]:
user_ids = train_json_fe.userid.apply(lambda x: user_id_map[x])
item_ids = train_json_fe.itemid.apply(lambda x: item_id_map[x])
prediction_lfm_fe = model_lfm_fe.predict(user_ids.values, item_ids.values, user_features=user_features, item_features=item_features)

In [ ]:
sklearn.metrics.roc_auc_score(train_json_fe.rating,prediction_lfm_fe)


In [ ]:
train_data, test_data = train_test_split(train,random_state=32, shuffle=True)


In [ ]:
ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])))



In [ ]:
train_data.shape, test_data.shape, ratings_coo.shape

## LightFm

In [ ]:
NUM_THREADS = 10 #число потоков
NUM_COMPONENTS = 41 #число параметров вектора 
NUM_EPOCHS = 14 #число эпох обучения

model = LightFM(learning_rate=0.12, loss='logistic',
                no_components=NUM_COMPONENTS)
model = model.fit(ratings_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

In [ ]:
#train_lfm, test_lfm = cross_validation.random_train_test_split(ratings_coo, test_percentage=0.25)
#model.fit(train, item_features=item_features, epochs=50)


In [ ]:
preds_LFM = model.predict(test_data.userid.values, test_data.itemid.values)

In [ ]:
sklearn.metrics.roc_auc_score(test_data.rating,preds_LFM)


In [ ]:
preds_LFM = model.predict(test.userid.values,
                      test.itemid.values)
normalized_preds = (preds_LFM - preds_LFM.min())/(preds_LFM - preds_LFM.min()).max()
submission['rating']= normalized_preds
submission.to_csv('/kaggle/working/submission_log.csv', index=False)

## SVD

In [ ]:
#create matrix for svd/als
from surprise import Reader, Dataset
reader= Reader(rating_scale=(0,1))
data = Dataset.load_from_df(train_data[['userid','itemid','rating']], reader) 

In [ ]:
svd = SVD()

In [ ]:
#Hyperparametr tuning
param_grid = {'n_epochs': [5, 20], 
              'lr_all': [0.002, 0.01],
              'reg_all': [0.1, 1]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
gs.fit(data)
params = gs.best_params['rmse']

model_svd = SVD(n_epochs=params['n_epochs'],lr_all=params['lr_all'], reg_all=params['reg_all'])

In [ ]:
#create model 
trainingSet = data.build_full_trainset()
model = model_svd.fit(trainingSet)

In [ ]:
pred_svd = model.test(zip(test.userid.values, test.itemid.values, [0]*len(test.userid.values)))
est = [i.est for i in pred_svd]
pred_sub = np.array(est)

In [ ]:
submission['rating']= pred_sub
submission.to_csv('/kaggle/working/submission_log.csv', index=False)

## BaselineOnly (ALS/SGD)

In [ ]:
# (https://surprise.readthedocs.io/en/stable/prediction_algorithms.html) - documentation

# ALS Algoritm
bsl_options = {'method': 'als',
               'n_epochs': [5,10],
               'reg_u': 15,
               'reg_i': 10
               }
algo = BaselineOnly(bsl_options=bsl_options)

In [ ]:
#learning model
trainingSet = data.build_full_trainset()
testSet = trainingSet.build_testset()
model_als = algo.fit(trainingSet)


In [ ]:
#pred test for submission
pred_als = model_als.test(zip(test.userid.values, test.itemid.values, [0]*len(test.userid.values)))

In [ ]:
#geting submission
est = [i.est for i in pred_als]
pred_sub = np.array(est)
submission['rating']= pred_sub
submission.to_csv('/kaggle/working/submission_log.csv', index=False)


In [ ]:
# SGD Algoritm
bsl_options = {'method': 'sgd',
               'n_epochs': 20,
               'reg': 0.000262055315534682,
               'learning_rate':0.0069826353714177785
               }

algo = BaselineOnly(bsl_options=bsl_options)


In [ ]:
#learning model
trainingSet = data.build_full_trainset()
testSet = trainingSet.build_testset()

algo = BaselineOnly(bsl_options=bsl_options)
model_sgd = algo.fit(trainingSet)

In [ ]:
pred_sgd = model_sgd.test(zip(test.userid.values, test.itemid.values, [0]*len(test.userid.values)))

In [ ]:
#geting submission
est = [i.est for i in pred_sgd]
pred_sub = np.array(est)
submission['rating']= pred_sub
submission.to_csv('/kaggle/working/submission_log.csv', index=False)


In [ ]:
pip install auto-surprise

In [ ]:
# tuning hyperparametrs with auto_surprise
from auto_surprise.engine import Engine
import hyperopt

engine = Engine(['svd', 'svdpp', 'knn_basic', 'knn_baseline'])
# Start the trainer
best_algo, best_params, best_score, tasks = engine.train_json(
data=data,
target_metric='test_rmse',
cpu_time_limit=60*60*2,
max_evals=100,
hpo_algo=hyperopt.tpe.suggest
)
# Build the model using the best algorithm and hyperparameters
best_model = engine.build_model(best_algo, best_params)

#defined the required parameters
#Best algorithm: baseline_only
#Best hyperparameters: {'bsl_options': {'learning_rate': 0.0069826353714177785, 'method': 'sgd', 'reg': 0.000262055315534682}}



## KKN(CF)

In [ ]:
train_data.itemid.nunique(), train_data.userid.nunique() 

In [ ]:
user_sample = train_data.userid.sample(8000)
item_sample = train_data.itemid.sample(2000)

In [ ]:
from surprise import KNNBasic
from surprise import Reader, Dataset

reader = Reader(rating_scale=(0, 1))
#data = Dataset.load_from_df(train_data[['userid', 'itemid', 'rating']], reader)

data = Dataset.load_from_df(train_data[train_data.userid.isin(user_sample) &
                              train_data.itemid.isin(item_sample)][['userid', 'itemid', 'rating']], reader)

sim_options = {
    'name': 'cosine',
    'user_based': False
}
 
knn = KNNBasic()
trainingSet = data.build_full_trainset()
model_knn = knn.fit(trainingSet)


In [ ]:
testSet = trainingSet.build_testset()
predictions = model_knn.test(testSet)

predict = model_knn.test(zip(test.userid.values, test.itemid.values, [0]*len(test.userid.values)))

In [ ]:
est = [i.est for i in predictions]
arr = np.array(est)

In [ ]:
sklearn.metrics.roc_auc_score(test_data.rating,preds)

In [ ]:
est = [i.est for i in preds]
arr = np.array(est)

# Выводы

1. EDA / FE
    - Структурное описание датасета приведено в разделе Overview блока EDA
    - Графическое и семантическое описание значений приведены в разделе Variables блока EDA
    - Анализ корреляций приведен в разделе Correlation блока EDA

1.1. Предобработка датасета включает:
Удаление лишних признаков
Графический и корреляционный анализ

1.2.Заключение по EDA: 
Заметных корреляций не обнаружено за исключением unixReviewTime и ItemID, verified и itemid. Попробуем на основе этих данных создать новые признаки и использовать взаимодействие с userid, itemid
Тестирование модели на датасете с удаленными признаками (исключая itemid, userid) практически дало схожую оценку(ROC/AUC)
Создали dummy переменные на основе признака brand

1.3.Создали два новых признака item_features на основе unixReviewTime и  user_features на основе verified. Однако полученный результат по метрике roc_auc сильно упал(до 0,52)

2. ML
ML(без обработки на основе базового train) 
Было использовано три модели: LightFM, SVD, ALS/SGD
Наилучший результат показала модель LightFM - 0.75, 
Другие результаты: SVD - 0.734, SGD - 0.737, ALS - 0.729
Модели обучались с подбором гиперпараметров.

ML(c обработкой на основе склеенного датасета) 


3. Протипирование должно показать реальные результаты работы моделей. 
не делалось

4. Рекомендации. 
Расширить FE. Побровать применить нейроные сетиюПопробовать другие метрики. 
Разработать прототип и проверить работу модели на реальных данных.

